In [1]:
import seaborn as sns
from pathlib import Path
from queries import*
from utilities import*
import os
from google.cloud import storage


In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\mdsp\\Desktop\\EPFL\\deep-melo-lts4-2ca7c864d933.json"

In [3]:
PATIENT_ID = '4A1'
base_directory = Path(Path.cwd()).parent
INPUT_PATH = os.path.join(base_directory,'data','input','DEEPMEL_' + PATIENT_ID + '\\')
OUTPUT_PATH = os.path.join(base_directory,'data','output','DEEPMEL_' + PATIENT_ID + '\\')
PLOT_PATH ='.\plots\DEEPMEL_' + PATIENT_ID +'\''
INPUT_FILE = 'DEEPMEL_'+ PATIENT_ID +'_cell_seg_data.txt'
TABLE_NAME_BQ = 'DEEPMEL_'+ PATIENT_ID +'_cell_seg_data'
#HIGH_VOLUME = high_volume_patients[PATIENT_ID]

In [4]:
nodes = pd.read_csv(INPUT_PATH + INPUT_FILE,sep='\t')

In [5]:
columns_to_keep = ['cell_id','cell_x_position','cell_y_position','tissue_category','phenotype']
nodes = nodes[columns_to_keep]

In [6]:
len(nodes)

142256

In [7]:
nodes.head(3)

,cell_id,cell_x_position,cell_y_position,tissue_category,phenotype
0,1,15246.8,51794.2,stroma,DAPIp
1,2,15216.1,51806.1,stroma,DAPIp
2,3,15235.4,51811.1,stroma,DAPIp


In [8]:
INPUT_FILE = 'DEEPMEL_'+ PATIENT_ID +'_cell_seg_data.csv'
nodes.to_csv(INPUT_PATH + INPUT_FILE, index=False)

In [9]:
#!gsutil cp  .data/input/DEEPMEL_3P1/DEEPMEL_3P1_cell_seg_data.csv  gs://deep-melo-lts4/DEEPMEL_3P1_cell_seg_data.csv

In [10]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client.from_service_account_json('C:\\Users\\mdsp\\Desktop\\EPFL\\deep-melo-lts4-2ca7c864d933.json')

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [11]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [13]:
Directory = Path(Path.cwd()).parent
Directory_path = os.path.join(Directory, 'data','input')
files = os.listdir(Directory_path)
count_phenotype = pd.DataFrame({})


for file in files[:]:
    
    array = file.split('_')
    if len(array)==2:
        condition = array[0]
        PATIENT_ID = array[1]
    else:
        condition == ''
        
    if condition == 'DEEPMEL':
        
        print('Treating patient' + PATIENT_ID)
        INPUT_PATH = os.path.join(Directory,'data','input','DEEPMEL_' + PATIENT_ID + '\\')
        INPUT_FILE = 'DEEPMEL_'+ PATIENT_ID +'_cell_seg_data.csv'
        FILE = os.path.join(INPUT_PATH,INPUT_FILE)
        URL = 'gs:/deep-melo-lts4/'+ 'DEEPMEL_'+ PATIENT_ID +'_cell_seg_data.csv'
        print(FILE)
        upload_blob('deep-melo-lts4', FILE, URL)

Treating patient1C5
C:\Users\mdsp\Desktop\EPFL\data\input\DEEPMEL_1C5\DEEPMEL_1C5_cell_seg_data.csv
File C:\Users\mdsp\Desktop\EPFL\data\input\DEEPMEL_1C5\DEEPMEL_1C5_cell_seg_data.csv uploaded to gs:/deep-melo-lts4/DEEPMEL_1C5_cell_seg_data.csv.
Treating patient1R2
C:\Users\mdsp\Desktop\EPFL\data\input\DEEPMEL_1R2\DEEPMEL_1R2_cell_seg_data.csv
File C:\Users\mdsp\Desktop\EPFL\data\input\DEEPMEL_1R2\DEEPMEL_1R2_cell_seg_data.csv uploaded to gs:/deep-melo-lts4/DEEPMEL_1R2_cell_seg_data.csv.
Treating patient1S2
C:\Users\mdsp\Desktop\EPFL\data\input\DEEPMEL_1S2\DEEPMEL_1S2_cell_seg_data.csv
File C:\Users\mdsp\Desktop\EPFL\data\input\DEEPMEL_1S2\DEEPMEL_1S2_cell_seg_data.csv uploaded to gs:/deep-melo-lts4/DEEPMEL_1S2_cell_seg_data.csv.
Treating patient2D2
C:\Users\mdsp\Desktop\EPFL\data\input\DEEPMEL_2D2\DEEPMEL_2D2_cell_seg_data.csv
File C:\Users\mdsp\Desktop\EPFL\data\input\DEEPMEL_2D2\DEEPMEL_2D2_cell_seg_data.csv uploaded to gs:/deep-melo-lts4/DEEPMEL_2D2_cell_seg_data.csv.
Treating pat

In [20]:
from google.cloud import bigquery


for file in files[:]:
    
    array = file.split('_')
    if len(array)==2:
        condition = array[0]
        PATIENT_ID = array[1]
    else:
        condition == ''
        
    if condition == 'DEEPMEL':
        
        bigqueryClient = bigquery.Client()
        jobConfig = bigquery.LoadJobConfig()
        jobConfig.skip_leading_rows = 1
        jobConfig.source_format = bigquery.SourceFormat.CSV
        jobConfig.write_disposition = bigquery.WriteDisposition.WRITE_APPEND   
        jobConfig.autodetect=True
        
        print('Creating a Bigquery table for a patient' + PATIENT_ID)
        datasetName = "deepmelo"
        targetTable = 'DEEPMEL_'+ PATIENT_ID +'_cell_seg_data'
        uri = "gs://deep-melo-lts4/gs:/deep-melo-lts4/DEEPMEL_" + PATIENT_ID + "_cell_seg_data.csv"
        tableRef = bigqueryClient.dataset(datasetName).table(targetTable)
        bigqueryJob = bigqueryClient.load_table_from_uri(uri, tableRef, job_config=jobConfig)
        bigqueryJob.result()

Creating a Bigquery table for a patient1C5
Creating a Bigquery table for a patient1R2
Creating a Bigquery table for a patient1S2
Creating a Bigquery table for a patient2D2
Creating a Bigquery table for a patient3K1
Creating a Bigquery table for a patient4A1
Creating a Bigquery table for a patient4B1
Creating a Bigquery table for a patient4D1
Creating a Bigquery table for a patient4E1
Creating a Bigquery table for a patient4F1
Creating a Bigquery table for a patient4G1
Creating a Bigquery table for a patient4H1
Creating a Bigquery table for a patient4I1
Creating a Bigquery table for a patient4J1
Creating a Bigquery table for a patient4K1
Creating a Bigquery table for a patient4L1
Creating a Bigquery table for a patient4N1
Creating a Bigquery table for a patient4O1
Creating a Bigquery table for a patient4P1
Creating a Bigquery table for a patient4Q1
Creating a Bigquery table for a patient4R1
Creating a Bigquery table for a patient4S1
Creating a Bigquery table for a patient4S1
Creating a 